In [85]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import requests
import re
import validators
from googletrans import Translator
from translate import Translator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [86]:
data = pd.read_csv("./Datathon_TrainData_org.csv")
data.head()
# final_test_data = pd.read_csv("./TestData_org.csv")

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,24-01-2000,Vladimir Putin,Speech at a conference of the heads of republi...,"Esteemed Mr Lebedev,\n\nLadies and Gentlemen:\...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,UPSET,OTHERS
1,USA,09-09-2021 00:00,Joe Biden,Remarks by President Biden on Fighting the COV...,"THE PRESIDENT: Good evening, my fellow Americ...",President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,OPTIMISM,DEVELOPMENT
2,Russia,24-12-2004,Vladimir Putin,Excerpts from the Opening Speech at a Session ...,"Good afternoon, dear colleagues,\n\nOur meetin...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
3,USA,14-08-2014,Barack Obama,\nStatement by the President,"Good afternoon, everybody. This sound system i...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,INTERNATIONAL AFFAIRS
4,USA,26-03-2021 00:00,Joe Biden,Statement by President Biden on the Attack on ...,More Americans voted in the 2020 elections tha...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,UPSET,OTHERS


In [87]:
x_train = data["Text_of_Speech"]
y_train = data["Emotion"]
final_pred = final_test_data["Text_of_Speech"]

In [88]:
stop_words = stopwords.words("english")

In [89]:
def remove_un(text):
    te = ""
    for i in text.split("\n"):
        if len(i) > 0:
            te = te + i 
    return te

In [90]:
def remove_un1(text):
    te = ""
    for i in text.split("\t"):
        if len(i) > 0:
            te = te + i 
    return te

In [91]:
final_pred = final_pred.apply(remove_un)
final_pred = final_pred.apply(remove_un1)

In [92]:
x_train = x_train.apply(remove_un)
x_train = x_train.apply(remove_un1)

In [93]:
final_pred

0      Good evening, everybody.  Please have a seat. ...
1      Colleagues, we are meeting out of schedule tod...
2      Now, one other thing I want to point out -- so...
3      Good afternoon, everyone.  I just had the oppo...
4       Well, I’ve got my team here to talk about Ebo...
                             ...                        
397    THE PRESIDENT:  Thank you.  Thank you very muc...
398    Good afternoon, dear colleagues,Today we are e...
399    Colleagues, welcome to this traditional meetin...
400     Hi everybody.  Right now, the United States S...
401    THE PRESIDENT:  Jocelyn, thank you.  Please.  ...
Name: Text_of_Speech, Length: 402, dtype: object

In [94]:
lem = WordNetLemmatizer()

In [95]:
def removing_stop_words(text):
    data = nltk.word_tokenize(text)
    te = ""
    for i in data:
        if i.strip().lower() not in stop_words:
            new = lem.lemmatize(i)
            te = te + new + " "
    return te
        

In [96]:
x_train = x_train.apply(removing_stop_words)

In [97]:
vectorizer = TfidfVectorizer()
X_train_count_matrix=vectorizer.fit_transform(x_train,y_train)

# new_model = LogisticRegression()
# new_model.fit(X_train_count_matrix, y_train)

In [69]:
final = VotingClassifier(estimators=[('logic', LogisticRegression()), ('SVC',SVC(probability=True)),("random",RandomForestClassifier())], voting='soft').fit(X_train_count_matrix, y_train)

In [82]:
X_test_count_matrix = vectorizer.transform(final_pred)

In [83]:
ypred4 = final.predict(X_train_count_matrix)

In [98]:
print(final.score(X_train_count_matrix,y_train))

49.563


In [233]:
with open('final_model.pkl', 'wb') as model_file:
    pickle.dump(final, model_file)

In [245]:
import csv
file_name = 'output.csv'

# Writing to CSV file
with open(file_name, mode='w', newline="") as file:
    writer = csv.writer(file)
#     writer.writerow(['Name'])
    writer.writerows([[item] for item in ypred4])